In [4]:
import os
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
from Model.utils import *
from Model.modules import UNet, UNet_conditional, EMA
import numpy as np
import logging
import copy


### Conditional cases of sampling of 8-class IPF regenerations

In [ ]:
from Model.ddpm_conditional import Diffusion
import os
import numpy as np
from PIL import Image

device = "cuda"
model = UNet_conditional(num_classes=8).to(device)
diffusion = Diffusion(img_size=64, device=device)
y = torch.Tensor([1] * 16).long().to(device) # Specify IPF class here [0-7]

# Define the checkpoint folders
checkpoint_folder = r"Pre-trained"

    # Load the checkpoint
checkpoint_path = os.path.join(checkpoint_folder, "ema_ckpt.pt")
ckpt = torch.load(checkpoint_path)
model.load_state_dict(ckpt)

# Generate 10 groups
for trail in range(10):
    x = diffusion.sample(model, 16, y, cfg_scale=3)
    x_np = x.cpu().numpy()
    output_folder = r"Specify\folder\you\want\to\save\regenerations"
    os.makedirs(output_folder, exist_ok=True)
    
    for i in range(16):
        image = x_np[i]
        image = np.transpose(image, (1, 2, 0))
        pil_image = Image.fromarray(np.uint8(image))
        image_path = os.path.join(output_folder, f"{trail}_{i+1}.png")
        pil_image.save(image_path)